In [54]:
import matplotlib.pyplot as plt
import pathlib
import requests
import io
import os
import numpy as np
import pathlib
import PIL
import base64

import tensorflow as tf
import cv2        as cv
import numpy      as np

from fastapi                  import FastAPI,File, UploadFile
from fastapi.middleware.cors  import CORSMiddleware
#from mushroom_learning.gcp    import get_model

from tensorflow               import keras
from tensorflow.keras         import utils


# Checking API

In [56]:
URL_hosted        = 'https://mushroom-docker-lpuaioudtq-ew.a.run.app/'
URL_hosted_size   = URL_hosted + 'size/'
URL_hosted_poison = URL_hosted + 'poison/'
URL_hosted_species= URL_hosted + 'species/'

URL_base    = 'http://127.0.0.1:8000/'
URL_size    = URL_base+"size/"
URL_predict = URL_base+"predict/"
URL_poison  = URL_base+"poison/"
URL_species = URL_base+"species/"

print(URL_size)
print(URL_predict)

http://127.0.0.1:8000/size/
http://127.0.0.1:8000/predict/


## Open an Image

To really test how it will be in production, we need to put this notebook somewhere it can see the image, but the api cannot.

In [2]:
!ls -rtlh 'amanita.jpg'
!pwd

-rwxr-xr-x 1 danielsptanner danielsptanner 85K Mar  9 11:35 amanita.jpg
/home/danielsptanner/code/DSP-Tan/mushroom_learning/notebooks


In [57]:
amanita_loc='amanita.jpg'
image=PIL.Image.open(amanita_loc)
print(type(image))
image_array = np.array(image)
print(image_array.shape)
print(f'So every row has 1024 columns, each with a depth of 3, total elements:{657*1024*3}')
print(f'\nThese are not bytes, as this does not bear a fixed relation to the filesize: {2018304/59836}')

<class 'PIL.JpegImagePlugin.JpegImageFile'>
(600, 800, 3)
So every row has 1024 columns, each with a depth of 3, total elements:2018304

These are not bytes, as this does not bear a fixed relation to the filesize: 33.73059696503777


## Check uvicorn run locally

### Check base URL

In [43]:
URL_base    = 'http://127.0.0.1:8000/'
URL_size    = URL_base+"size/"
URL_predict = URL_base+"predict/"

In [63]:
#response = requests.get(URL_base)
#print(response.json())
response = requests.get(URL_hosted)
print(response.json())

["Don't eat that mushroom!"]


### Check image size

#### Open image as binary, write to np array byte array, and encode as base64

In [64]:
! ls -rtlh amanita.jpg 
with open('amanita.jpg', 'rb') as f:
    im_API = f.read()
im_API=np.asarray(bytearray(im_API), dtype="uint8")
encoded = base64.b64encode(im_API)

-rwxr-xr-x 1 danielsptanner danielsptanner 85K Mar  9 11:35 amanita.jpg


#### Confirm we can decode this and repeat preprocessing steps

In [65]:
decoded_mush=base64.decodebytes(encoded)
print(f'filesize {len(decoded_mush)/1000}, type of array {type(decoded_mush)}')
new_im_API=np.asarray(bytearray(decoded_mush), dtype="uint8")
np.array_equal(new_im_API,im_API)

filesize 86.717, type of array <class 'bytes'>


True

### Pass encoded array to API /size end point

In [32]:
print(URL_size)
URL_hosted_size=URL_hosted+'size/'
print(URL_hosted_size)

http://127.0.0.1:8000/size/
https://mushroom-docker-lpuaioudtq-ew.a.run.app/size/


In [75]:
files={'mush':encoded}
response = requests.post(URL_size,data=files)
print(response)

response = requests.post(URL_species,data=files)
print(response.json())

response = requests.post(URL_poison,data=files)
print(response.json())

#response = requests.post(URL_hosted_size,data=files)
#print(response)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /size/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f306f092520>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [76]:
response = requests.post(URL_hosted,data=files)
print(response)

<Response [405]>


### Pass encoded array to API /poison end point

In [10]:
files={'mush':encoded}
response = requests.get(URL_poison,data=files)
print(response)
print(response.json())

<Response [200]>
This mushroom is most likely poisonous. Score: 0.70


In [11]:
def check_size(mush: bytes = File(...)):
    # convert to bytes with bytearray, and to np array
    image = np.asarray(bytearray(mush), dtype="uint8")
    return f'This file is {len(image)/1000} Kbytes'

In [12]:
img = PIL.Image.open(amanita_loc, mode='r')
print(type(img))
img_byte_arr = io.BytesIO()
img.save(img_byte_arr, format='jpeg')
img_byte_arr = img_byte_arr.getvalue()
print(len(img_byte_arr)/1000)
print(check_size(img_byte_arr))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
76.058
This file is 76.058 Kbytes


In [ ]:
!ls -rtlh amanita.jpg

## Check local model prediction

In [14]:
model=keras.models.load_model('../our_first_model/')

In [15]:
def image_API(im_path,size):
    # convert to bites (this step should be excluded in an actual API, as the
    # image arrives in bytes from the get or post functions.)
    with open(im_path, 'rb') as f:
        im_API = f.read()
    # convert to np array.    
    im_API=np.asarray(bytearray(im_API), dtype="uint8")
    # decode byte array back into image, and then adjust
    # for cv's automatic BGR representation
    im_API = cv.imdecode(im_API,cv.IMREAD_COLOR)
    im_API = cv.cvtColor(im_API , cv.COLOR_BGR2RGB)
    
    # resize using tensor flow with nearest neighbour interpolation
    im_API=tf.image.resize(im_API,size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # Expand for CNN
    im_API = tf.expand_dims(im_API, 0) 
    
    return im_API

In [16]:
im_API=image_API(amanita_loc,(224,224))

In [24]:
prediction_API=model.predict(im_API)
classif_API = int(prediction_API > .5)
print(f'{prediction_API[0][0]:.2f}:{("poison","edible")[classif_API]}')

0.70:edible


## Check local docker incapsulated API

In [46]:
files={'mush':encoded}
response = requests.post(URL_poison,data=files)
print(response)
print(response.json())

<Response [200]>
This mushroom is most likely poisonous. Score: 0.70


## Check google image registry deployed docker


In [47]:
URL_base='https://mushroom-docker-lpuaioudtq-ew.a.run.app/'
URL_size    = URL_base+"size/"
URL_predict = URL_base+"predict/"
URL_poison  = URL_base+"poison/"
URL_species = URL_base+"species/"

In [50]:
files={'mush':encoded}
response = requests.get(URL_base)

print(response.json())

<Response [405]>
["Don't eat that mushroom!"]


In [53]:
response=requests.post(URL_size,data=files)
print(response.json())

{'detail': 'Method Not Allowed'}
